In [9]:
import mysql.connector
import pandas as pd
import logging
from datetime import datetime

# -------------------------------
# تنظیمات
# -------------------------------
EXCEL_FILE = "abnormal_results_mhi_lube_oil_g11.xlsx"
LOG_FILE = "export_abnormal.log"

# تنظیم لاگ
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, encoding='utf-8'),
        logging.StreamHandler()
    ]
)

# -------------------------------
# 1. اتصال به MySQL
# -------------------------------
def get_connection():
    return mysql.connector.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        password='',
        database='dsas'
    )

# -------------------------------
# 2. تابع اصلی: استخراج و ذخیره Abnormal
# -------------------------------
def export_abnormal_to_excel():
    conn = None
    try:
        conn = get_connection()
        query = """
            SELECT 
                `id`, `AssetID_8341`, `AssetID_8342`, `AssetID_8343`, `AssetID_8344`,
                `AssetID_8346`, `AssetID_9286`, `AssetID_9287`, `unitID`,
                `model_name`, `model_result`, `model_value`,
                `DateTime`, `RecordDate`, `RecordTime`, `TimeStamps`,
                `created_at`, `updated_at`
            FROM `results_table_mhi_lube_oil_g11`
            WHERE `model_result` = 'Abnormal'
            ORDER BY `TimeStamps` DESC
        """
        
        logging.info("در حال خواندن رکوردهای Abnormal از جدول...")
        df = pd.read_sql(query, conn)

        if df.empty:
            logging.info("هیچ رکوردی با model_result = 'Abnormal' یافت نشد.")
            print("هیچ مورد غیرعادی یافت نشد.")
            return

        # ذخیره در اکسل
        df.to_excel(EXCEL_FILE, index=False)
        logging.info(f"{len(df)} رکورد Abnormal با موفقیت در '{EXCEL_FILE}' ذخیره شد.")
        print(f"موفقیت‌آمیز: {len(df)} مورد غیرعادی در '{EXCEL_FILE}' ذخیره شد.")

    except mysql.connector.Error as e:
        logging.error(f"خطا در اتصال به دیتابیس یا اجرای کوئری: {e}")
        print(f"خطا در دیتابیس: {e}")
    except Exception as e:
        logging.error(f"خطای غیرمنتظره: {e}")
        print(f"خطای غیرمنتظره: {e}")
    finally:
        if conn and conn.is_connected():
            conn.close()
            logging.info("اتصال به دیتابیس بسته شد.")

# -------------------------------
# 3. اجرا
# -------------------------------
if __name__ == "__main__":
    start_time = datetime.now()
    logging.info("شروع استخراج رکوردهای Abnormal...")
    print("در حال استخراج رکوردهای غیرعادی...")
    
    export_abnormal_to_excel()
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    logging.info(f"پایان فرآیند. مدت زمان: {duration:.2f} ثانیه")
    print(f"تمام شد. مدت زمان: {duration:.2f} ثانیه")

2025-11-14 15:52:47,320 | INFO | شروع استخراج رکوردهای Abnormal...
2025-11-14 15:52:47,324 | INFO | در حال خواندن رکوردهای Abnormal از جدول...
C:\Users\pishva_r\AppData\Local\Temp\ipykernel_16308\2598748301.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
2025-11-14 15:52:47,397 | INFO | 1 رکورد Abnormal با موفقیت در 'abnormal_results_mhi_lube_oil_g11.xlsx' ذخیره شد.
2025-11-14 15:52:47,399 | INFO | اتصال به دیتابیس بسته شد.
2025-11-14 15:52:47,401 | INFO | پایان فرآیند. مدت زمان: 0.08 ثانیه


در حال استخراج رکوردهای غیرعادی...
موفقیت‌آمیز: 1 مورد غیرعادی در 'abnormal_results_mhi_lube_oil_g11.xlsx' ذخیره شد.
تمام شد. مدت زمان: 0.08 ثانیه
